In [1]:
#Primeiro vamos importar algumas bibliotecas que posso usar
#Libraries
from haversine import haversine
import plotly.express as px
import plotly.graph_objects as go

#Bibliotecas necessárias
import pandas as pd
import streamlit as st
import numpy as np
from PIL import Image
import folium
from streamlit_folium import folium_static
import inflection

In [2]:
#Vamos ler o arquivo
df=pd.read_csv( "zomato.csv" )

In [3]:
df1 = df.copy()

In [4]:
#Aqui vamos renomear os nomes das colunas, deixando tudo maíusculo e onde tem espaço substituir por _
#Função
def rename_columns(df1):
    title = lambda x: inflection.titleize(x)
    
    snakecase = lambda x: inflection.underscore(x)
    
    spaces = lambda x: x.replace(" ", "")
    
    cols_old = list(df1.columns)
    
    cols_old = list(map(title, cols_old))
    
    cols_old = list(map(spaces, cols_old))
    
    cols_new = list(map(snakecase, cols_old))
    
    df1.columns = cols_new
    
    return df1
#Aqui temos que chamar a função, assim acontece a substituição dos nomes
rename_columns(df1)

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7522,5912546,Eataly,208,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.017326,41.065322,"Italian, Pizza, Fresh Fish",...,Turkish Lira(TL),0,0,0,0,4,4.3,5BA829,Very Good,1367
7523,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,208,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052620,41.050280,"Fast Food, Izgara, Seafood, Tea, Coffee",...,Turkish Lira(TL),0,0,0,0,2,4.5,3F7E00,Excellent,1172
7524,5923535,Boon Cafe & Restaurant,208,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052623,41.050717,Restaurant Cafe,...,Turkish Lira(TL),0,0,0,0,4,4.2,5BA829,Very Good,1160
7525,5914190,Kanaat Lokantası,208,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.016590,41.025741,"Home-made, Izgara",...,Turkish Lira(TL),0,0,0,0,3,4.0,5BA829,Very Good,770


In [5]:
#Aqui vamos alterar o nome dos Países para o código deles.
#Função
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}
def country_name(country_id):
    return COUNTRIES[country_id]

In [6]:
#Criação do Tipo de Categoria de Comida
#Função
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"


In [7]:
#Criação do tipo das cores, nada mais do que substituir os códigos pelos nomes das cores.
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(rating_color):
    return COLORS[rating_color]

In [8]:
#Devemos aplicar o .apply(country_name) para finalizar a função que criamos anteriormente.
df1["country_code"] = df1["country_code"].apply(country_name)
#Devemos aplicar o .apply(create_price_tye) para finalizar a função que criamos anteriormente.
df1["price_range"] = df1["price_range"].apply(create_price_tye)
#Devemos aplicar o .apply(color_name) para finalizar a função que criamos anteriormente.
df1["rating_color"] = df1["rating_color"].apply(color_name)

In [9]:
#Aqui categorizamos os restaurantes por só UM tipo de comida, o exercício pede assim
df1["cuisines"] = df1.loc[:, "cuisines"].astype(str).apply(lambda x: x.split(",")[0])

In [10]:
df1.head(1)

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,currency,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,expensive,4.6,darkgreen,Excellent,619


Descrição das colunas

Column	                       Description
Restaurant ID	               ID do restaurante
Restaurant Name	               Nome do Restaurante
Country Code	               Código do País
City	                       Nome da Cidade onde o restaurante está
Address	                       Endereço do restaurante
Locality	                   Localização e pontos de referência do restaurante
Locality Verbose	           Localização e pontos de referência do restaurante (Mais informações)
Longitude	                   Ponto geográfico de Longitude do Restaurante
Latitude	                   Ponto geográfico de Latitude do Restaurante
Cuisines	                   Tipos de Culinária servidos no restaurante
Average Cost for two	       Preço Médio de um prato para duas pessoas no restaurante
Currency	                   Moeda do país
Has Table booking	           Se o restaurante possui serviços de reserva; 1 - Sim; 0 - Não
Has Online delivery	           Se o restaurante possui serviços de pedido on-line; 1 - Sim; 0 - Não
Is delivering now	           Se o restaurante faz entregas; 1 - Sim; 0 - Não
Switch to order menu	-
Price range	                   Variação de preços do restaurante; 1 a 4 - Quanto maior o valor, mais caro serão os pratos
Aggregate rating	           Nota média do restaurante
Rating color	               Código Hexadecimal da cor do restaurante com base em sua nota média
Rating text	                   Categoria em que o restaurante está com base em sua nota média
Votes	                       Quantidade de avaliações que o restaurante já recebeu

In [11]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes'],
      dtype='object')

In [12]:
#Aqui vou verificar se há alguma coluna com somente 1 dado, nesse caso eu substituo o nome da coluna por outra e verifico
#Nesse caso a única coluna que possui somente 1 dado é a coluna abaixo, então vou remove-la.

valores_unicos = df1["switch_to_order_menu"].unique()
print(valores_unicos)

[0]


In [13]:
#Aqui vamos fazer a exclusão dessa coluna com valores únicos
df1 = df1.drop("switch_to_order_menu", axis=1)

In [14]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'price_range',
       'aggregate_rating', 'rating_color', 'rating_text', 'votes'],
      dtype='object')

In [15]:
#Aqui vamos verificar se há valores duplicados
df1.duplicated().sum()

585

In [16]:
#Como acima vimos que há valores duplicados, vamos exclui-los
df1 = df1.drop_duplicates().reset_index()
del df1["index"]

In [17]:
df1.duplicated().sum()

0

In [18]:
#Aqui vamos verificar se há valores NaN.
#Como não há valores NaN, não precisamos fazer a retirada deles.
df1.isnull().sum()

restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
dtype: int64

In [19]:
#daqui pra baixo vamos criar uma tabela estatística descritiva
def get_numerical_attributes(df1):
    return df1.select_dtypes(include=['int64', 'float64'])

In [20]:
def get_first_order_statistics(df1):
    # Central Tendency Metrics
    mean = pd.DataFrame(df1.apply(np.mean)).T
    median = pd.DataFrame(df1.apply(np.median)).T

    # Dispersion Metrics
    min_ = pd.DataFrame(df1.apply(min)).T
    max_ = pd.DataFrame(df1.apply(max)).T
    range_ = pd.DataFrame(df1.apply(lambda x: x.max() - x.min())).T
    std = pd.DataFrame(df1.apply(np.std)).T
    skew = pd.DataFrame(df1.apply(lambda x: x.skew())).T
    kurtosis = pd.DataFrame(df1.apply(lambda x: x.kurtosis())).T

    # Metrics Concatenation
    m = pd.concat([min_, max_, range_, mean, median, std, skew, kurtosis]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    
    return m

In [21]:
numerical_data = get_numerical_attributes(df1)

In [22]:
get_first_order_statistics(numerical_data)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,restaurant_id,549.000000,1.904028e+07,1.903973e+07,1.076193e+07,7.900186e+06,7.037181e+06,-0.142004,-1.714148
1,longitude,-122.700455,1.753106e+02,2.980110e+02,3.144123e+01,7.375862e+01,7.905341e+01,-0.614339,-0.734406
2,latitude,-41.330428,5.597698e+01,9.730741e+01,1.943494e+01,2.527002e+01,2.349819e+01,-1.075934,0.549946
3,average_cost_for_two,0.000000,2.500002e+07,2.500002e+07,7.506579e+03,2.500000e+02,3.022297e+05,81.500479,6737.976131
4,has_table_booking,0.000000,1.000000e+00,1.000000e+00,6.050130e-02,0.000000e+00,2.384133e-01,3.687661,11.602188
5,has_online_delivery,0.000000,1.000000e+00,1.000000e+00,3.497551e-01,0.000000e+00,4.768925e-01,0.630236,-1.603265
6,is_delivering_now,0.000000,1.000000e+00,1.000000e+00,1.745895e-01,0.000000e+00,3.796156e-01,1.714792,0.940783
7,aggregate_rating,0.000000,4.900000e+00,4.900000e+00,4.126765e+00,4.200000e+00,6.747053e-01,-3.941552,21.161428
8,votes,0.000000,4.133300e+04,4.133300e+04,6.043840e+02,3.280000e+02,1.115664e+03,11.382713,291.559110


In [23]:
df1.head(1)

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,average_cost_for_two,currency,has_table_booking,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes
0,6310675,Mama Lou's Italian Kitchen,Philippines,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,1100,Botswana Pula(P),1,0,0,expensive,4.6,darkgreen,Excellent,619


## Geral

1. Quantos restaurantes únicos estão registrados?

In [24]:
df1["restaurant_id"].nunique()

6942

2. Quantos países únicos estão registrados?


In [25]:
df1["country_code"].nunique()

15

3. Quantas cidades únicas estão registradas?

In [26]:
df1["city"].nunique()

125

4. Qual o total de avaliações feitas?

In [150]:
aff = df1["votes"].sum()
f"{aff:,}".replace(",", ".")

'4.195.634'

5. Qual o total de tipos de culinária registrados

In [28]:
df1["cuisines"].nunique()

166

## País

1. Qual o nome do país que possui mais cidades registradas?


In [29]:
(df1[["country_code", "city"]].groupby(["country_code"])
                              .nunique()
                              .sort_values("city", ascending=False)
                              .reset_index()).iloc[0,0]

'India'

2. Qual o nome do país que possui mais restaurantes registrados

In [30]:
(df1[["country_code", "restaurant_id"]].groupby(["country_code"])
                                       .nunique()
                                       .sort_values("restaurant_id", ascending=False)
                                       .reset_index()).iloc[0,0]

'India'

3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4
registrados?


In [31]:
(df1.loc[df1["price_range"] == "gourmet", ["country_code", "restaurant_id"]]
                                          .groupby(["country_code"])
                                          .count()
                                          .sort_values("restaurant_id", ascending=False)
                                          .reset_index()).iloc[0,0]

'United States of America'

4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [32]:
(df1[["country_code", "cuisines"]].groupby(["country_code"])
                                 .nunique()
                                 .sort_values("cuisines", ascending=False)
                                 .reset_index()).iloc[0,0]

'India'

5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [33]:
(df1[["country_code", "votes" ]].groupby(["country_code"])
                                .sum()
                                .sort_values("votes", ascending=False)
                                .reset_index()).iloc[0,0]

'India'

6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [34]:
(df1.loc[df1["is_delivering_now"] == 1, ["country_code", "restaurant_id"]].groupby(["country_code"])
                                                                          .count()
                                                                          .sort_values("restaurant_id", ascending=False)
                                                                          .reset_index())

,country_code,restaurant_id
0,India,1150
1,United Arab Emirates,52
2,Philippines,7
3,Qatar,3


7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [35]:
(df1.loc[df1["has_table_booking"] == 1, ["country_code", "restaurant_id"]].groupby(["country_code"])
                                                                          .count()
                                                                          .sort_values("restaurant_id", ascending=False)
                                                                          .reset_index()).iloc[0,0]

'India'

8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [36]:
df1[["country_code", "votes"]].groupby(["country_code"]).mean().sort_values("votes", ascending=False).reset_index().iloc[0,0]

'Indonesia'

9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [37]:
(df1[["country_code", "aggregate_rating"]].groupby(["country_code"])
                                         .mean()
                                         .sort_values("aggregate_rating", ascending=False)
                                         .reset_index().iloc[0,0])

'Indonesia'

10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [38]:
(df1[["country_code", "aggregate_rating"]].groupby(["country_code"])
                                         .mean()
                                         .sort_values("aggregate_rating", ascending=True)
                                         .reset_index().iloc[0,0])

'Brazil'

11. Qual a média de preço de um prato para dois por país?

In [39]:
(round (df1[["country_code", "average_cost_for_two"]].groupby(["country_code"])
                                                    .mean()
                                                    .sort_values("average_cost_for_two", ascending=False), 2)
                                                    .reset_index())

,country_code,average_cost_for_two
0,Indonesia,303000.00
1,Australia,138959.78
2,Sri Lanka,2579.38
3,Philippines,1227.82
4,India,703.60
5,South Africa,339.23
6,Qatar,174.00
7,United Arab Emirates,153.72
8,Singapure,141.44
9,Brazil,138.81


## Cidade

1. Qual o nome da cidade que possui mais restaurantes registrados?

In [40]:
(df1[["city", "restaurant_id"]].groupby("city")
                               .count()
                               .sort_values("restaurant_id", ascending=False)
                               .reset_index()).iloc[0,0]

'Abu Dhabi'

2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [41]:
(df1.loc[df1["aggregate_rating"] >=4, ["city", "restaurant_id"]].groupby(["city"])
                                                                .count()
                                                                .sort_values("restaurant_id", ascending=False)
                                                                .reset_index()).iloc[0,0]

'London'

3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [42]:
(df1.loc[df1["aggregate_rating"] <=2.5, ["city", "restaurant_id"]].groupby(["city"])
                                                                  .count()
                                                                  .sort_values("restaurant_id", ascending=False)
                                                                  .reset_index()).iloc[0,0]

'Gangtok'

4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [43]:
(df1[["city", "average_cost_for_two"]].groupby(["city"])
                                      .mean()
                                      .sort_values("average_cost_for_two", ascending=False)
                                      .reset_index().iloc[0,0])

'Adelaide'

5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [44]:
(df1[["city", "cuisines"]].groupby(["city"])
                          .nunique()
                          .sort_values("cuisines", ascending=False)
                          .reset_index().iloc[0,0])

'Birmingham'

6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [45]:
(df1.loc[df1["has_table_booking"] == 1, ["city", "restaurant_id"]].groupby(["city"])
                                                                  .count()
                                                                  .sort_values("restaurant_id", ascending=False)
                                                                  .reset_index()).iloc[0,0]

'Bangalore'

7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [46]:
(df1.loc[df1["is_delivering_now"] == 1, ["city", "restaurant_id"]].groupby(["city"])
                                                                  .count()
                                                                  .sort_values("restaurant_id", ascending=False)
                                                                  .reset_index()).iloc[0,0]

'Vadodara'

8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [47]:
(df1.loc[df1["has_online_delivery"] == 1, ["city", "restaurant_id"]].groupby(["city"])
                                                                    .count()
                                                                    .sort_values("restaurant_id", ascending=False)
                                                                    .reset_index()).iloc[0,0]

'Bhopal'



## Restaurantes

1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [48]:
(df1[["restaurant_name", "votes"]].groupby(["restaurant_name"])
                                  .max()
                                  .sort_values("votes", ascending=False)
                                  .reset_index()).iloc[0,0]

'Bawarchi'

2. Qual o nome do restaurante com a maior nota média?

In [49]:
(df1[["restaurant_name", "restaurant_id", "aggregate_rating"]].groupby(["restaurant_name"])
                                  .max()
                                  .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[False, True])
                                  .reset_index()).iloc[0,0]

'Indian Grill Room'

3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [50]:
(df1[["restaurant_name", "average_cost_for_two"]].groupby(["restaurant_name"])
                                  .max()
                                  .sort_values("average_cost_for_two", ascending=False)
                                  .reset_index()).iloc[0,0]

"d'Arry's Verandah Restaurant"

4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [51]:
(df1.loc[df1["cuisines"] == "Brazilian", ["restaurant_name", "aggregate_rating", "restaurant_id"]]
                                         .groupby(["restaurant_name"])
                                         .min()
                                         .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[True, True])
                                         .reset_index()).iloc[0,0]

'Loca Como tu Madre'

5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [52]:
tipo_culinaria = (df1["cuisines"] == "Brazilian") & (df1["country_code"] == "Brazil")

(df1.loc[tipo_culinaria , ["restaurant_name", "aggregate_rating", "restaurant_id"]]
                                         .groupby(["restaurant_name"])
                                         .max()
                                         .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[False, True])
                                         .reset_index()).iloc[0,0]

'Braseiro da Gávea'

6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [53]:
(df1[["has_online_delivery", "votes"]].groupby(["has_online_delivery"])
                                      .mean()
                                      .sort_values("votes", ascending=False)
                                      .reset_index())

,has_online_delivery,votes
0,1,838.821664
1,0,478.284227


7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [54]:
(df1[["has_table_booking", "average_cost_for_two"]].groupby(["has_table_booking"])
                                                   .mean()
                                                   .sort_values("average_cost_for_two", ascending=False)
                                                   .reset_index())

,has_table_booking,average_cost_for_two
0,1,69998.423810
1,0,3482.264949


8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [55]:
tipo_culinaria = (df1["country_code"] == "United States of America") & ((df1["cuisines"] == "BBQ") | (df1["cuisines"] == "Japanese"))

(df1.loc[tipo_culinaria, ["average_cost_for_two", "cuisines"]].groupby(["cuisines"]).mean().sort_values("average_cost_for_two", ascending=False).reset_index())

,cuisines,average_cost_for_two
0,Japanese,56.406250
1,BBQ,39.642857


## Tipos de Culinária

1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [56]:
(df1.loc[df1["cuisines"] == "Italian", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                       .groupby(["restaurant_name"])
                                       .max()
                                       .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[False, True])
                                       .reset_index()).iloc[0,0]

'Cafe Del Sol Classico'

2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [57]:
(df1.loc[df1["cuisines"] == "Italian", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                       .groupby(["restaurant_name"])
                                       .min()
                                       .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[True, True])
                                       .reset_index()).iloc[0,0]

'Avenida Paulista'

3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [58]:
(df1.loc[df1["cuisines"] == "American", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                       .groupby(["restaurant_name"])
                                       .max()
                                       .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[False, True])
                                       .reset_index()).iloc[0,0]

'Burger & Lobster'

4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [59]:
(df1.loc[df1["cuisines"] == "American", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                       .groupby(["restaurant_name"])
                                       .min()
                                       .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[True, True])
                                       .reset_index()).iloc[0,0]

'Alston Bar & Beef'

5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [60]:
(df1.loc[df1["cuisines"] == "Arabian", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                      .groupby(["restaurant_name"])
                                      .max()
                                      .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[False, True])
                                      .reset_index()).iloc[0,0]

'Mandi@36'

6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [61]:
(df1.loc[df1["cuisines"] == "Arabian", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                      .groupby(["restaurant_name"])
                                      .min()
                                      .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[True, True])
                                      .reset_index()).iloc[0,0]

'Raful'

7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [62]:
(df1.loc[df1["cuisines"] == "Japanese", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                        .groupby(["restaurant_name"])
                                        .max()
                                        .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[False, True])
                                        .reset_index()).iloc[0,0]

'Sushi Samba'

8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [63]:
(df1.loc[df1["cuisines"] == "Japanese", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                        .groupby(["restaurant_name"])
                                        .min()
                                        .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[True, True])
                                        .reset_index()).iloc[0,0]

'Banzai Sushi'

9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [64]:
(df1.loc[df1["cuisines"] == "Home-made", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                         .groupby(["restaurant_name"])
                                         .max()
                                         .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[False, True])
                                         .reset_index()).iloc[0,0]

'Kanaat Lokantası'

10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [65]:
(df1.loc[df1["cuisines"] == "Home-made", ["restaurant_name", "restaurant_id", "aggregate_rating"]]
                                         .groupby(["restaurant_name"])
                                         .min()
                                         .sort_values(by=["aggregate_rating", "restaurant_id"], ascending=[True, True])
                                         .reset_index()).iloc[0,0]

'GurMekan Restaurant'

11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [66]:
(df1[['cuisines', 'average_cost_for_two']].groupby(['cuisines'])
                                         .max()
                                         .sort_values('average_cost_for_two', ascending=False)
                                         .reset_index()).head(1)

,cuisines,average_cost_for_two
0,Modern Australian,25000017


12. Qual o tipo de culinária que possui a maior nota média?

In [67]:
(df1[['cuisines', 'restaurant_id', 'aggregate_rating']].groupby(['cuisines'])
.max()
.sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True])
.reset_index()).iloc[0,0]

'Patisserie'

13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [68]:
pedidos_online_entrega = (df1['has_online_delivery'] == 1) & (df1['is_delivering_now'] == 1)

(df1.loc[pedidos_online_entrega, ['cuisines', 'restaurant_id']].groupby(['cuisines'])
                                                              .count()
                                                              .sort_values('restaurant_id', ascending=False)
                                                              .reset_index()).head(1)

,cuisines,restaurant_id
0,North Indian,317


## Perguntas Streamlit

### Visão País

1 - Quantidade de restaurantes registrados por país

In [69]:
(df1[["restaurant_id", "country_code"]]
 .groupby("country_code")
 .nunique()
 .sort_values("restaurant_id", ascending=False)
 .reset_index()
)

,country_code,restaurant_id
0,India,3120
1,United States of America,1378
2,England,400
3,South Africa,346
4,United Arab Emirates,300
5,Brazil,240
6,New Zeland,239
7,Australia,180
8,Canada,180
9,Turkey,159


2 - Quantidade de cidades registradas por país

In [70]:
(df1[["country_code", "city"]]
.groupby(["country_code"])
.nunique()
.sort_values("city", ascending=False)
.reset_index()
)

,country_code,city
0,India,49
1,United States of America,22
2,Philippines,12
3,South Africa,12
4,England,5
5,New Zeland,4
6,United Arab Emirates,4
7,Australia,3
8,Brazil,3
9,Canada,3


3 - Média de Avaliação feitas por País

In [71]:
round(df1[["country_code", "votes"]]
      .groupby(["country_code"])
      .mean()
      .sort_values("votes", ascending=False)
      .reset_index(),2
)

,country_code,votes
0,Indonesia,1112.82
1,India,897.50
2,Australia,724.21
3,Turkey,630.14
4,Philippines,604.98
5,United Arab Emirates,591.88
6,Canada,583.43
7,United States of America,379.84
8,Qatar,376.32
9,South Africa,236.82


4 - Média de Preço de um prato para duas pessoas por País

In [72]:
round(df1[["country_code", "average_cost_for_two"]]
      .groupby(["country_code"])
      .mean()
      .sort_values("average_cost_for_two", ascending=False)
      .reset_index(),2
)

,country_code,average_cost_for_two
0,Indonesia,303000.00
1,Australia,138959.78
2,Sri Lanka,2579.38
3,Philippines,1227.82
4,India,703.60
5,South Africa,339.23
6,Qatar,174.00
7,United Arab Emirates,153.72
8,Singapure,141.44
9,Brazil,138.81


### Visão Cidades

1 - Top 10 cidades com mais restaurantes na base de dados

In [73]:
(df1[['restaurant_id', 'country_code', 'city']]
.groupby(['city', 'country_code'])
.count()
.sort_values(['restaurant_id', 'city'] , ascending=[False,True])
.reset_index()
)

,city,country_code,restaurant_id
0,Abu Dhabi,United Arab Emirates,80
1,Agra,India,80
2,Allahabad,India,80
3,Amritsar,India,80
4,Ankara,Turkey,80
...,...,...,...
120,Marikina City,Philippines,1
121,Muntinlupa City,Philippines,1
122,Roodepoort,South Africa,1
123,San Juan City,Philippines,1


2 - Top 7 Cidades com Restaurantes com média de avaliação acima de 4

In [74]:
(df1.loc[df1["aggregate_rating"] >= 4, ["city", "country_code", "restaurant_id"]]
.groupby(["city", "country_code"])
.count()
.sort_values(["restaurant_id", "city"], ascending=[False,True])
.reset_index()
)

,city,country_code,restaurant_id
0,London,England,80
1,Bangalore,India,79
2,Houston,United States of America,77
3,Pune,India,77
4,Kolkata,India,76
...,...,...,...
116,Muntinlupa City,Philippines,1
117,Nasik,India,1
118,Roodepoort,South Africa,1
119,San Juan City,Philippines,1


3 - Top 7 Cidades com Restaurantes com média de avaliação abaixo de 2.5"

In [75]:
(df1.loc[df1["aggregate_rating"] <= 2.5, ["city", "country_code", "restaurant_id"]]
.groupby(["city", "country_code"])
.count()
.sort_values(["restaurant_id", "city"], ascending=[False,True])
.reset_index()
)

,city,country_code,restaurant_id
0,Gangtok,India,40
1,Ooty,India,19
2,Brasília,Brazil,16
3,São Paulo,Brazil,16
4,Rio de Janeiro,Brazil,13
5,Manchester,England,7
6,Clarens,South Africa,4
7,Shimla,India,4
8,Ankara,Turkey,3
9,Hamilton,New Zeland,3


4 - Top 10 Cidades mais restaurantes com tipos culinários distintos

In [89]:
(df1[["city", "country_code", "cuisines"]]
.groupby(["city", "country_code"])
.nunique()
.sort_values(["cuisines", "city"], ascending=[False, True])
.reset_index()
) 

,city,country_code,cuisines
0,Birmingham,England,32
1,Doha,Qatar,31
2,Houston,United States of America,30
3,Manchester,England,30
4,Montreal,Canada,30
...,...,...,...
120,Marikina City,Philippines,1
121,Muntinlupa City,Philippines,1
122,Roodepoort,South Africa,1
123,San Juan City,Philippines,1


### Visão Cozinha

1 - Melhores Restaurantes dos Principais tipos Culinários

In [106]:
df1[["restaurant_id", "cuisines", "country_code", "aggregate_rating"]].groupby(["cuisines", "country_code"]).max().sort_values(["aggregate_rating", "restaurant_id"], ascending=[False,True]).reset_index()

,cuisines,country_code,restaurant_id,aggregate_rating
0,Greek,Turkey,5915409,4.9
1,Patisserie,Turkey,6003426,4.9
2,Pizza,Turkey,6005865,4.9
3,Taiwanese,England,6127163,4.9
4,Seafood,New Zeland,7100916,4.9
...,...,...,...,...
594,Chinese,Brazil,6600495,0.0
595,Asian,Brazil,6703191,0.0
596,Vegetarian,Brazil,7303564,0.0
597,Drinks Only,South Africa,16547518,0.0


2 - Top 10 Restaurantes

In [127]:
#É apenas uma simulação de como faz, o resultado correto está na visão cozinha

cuisines = df1.loc[:, "cuisines"].unique().tolist()


countries = df1.loc[:, "country_code"].unique().tolist()
    
cols = [
        "restaurant_id",
        "restaurant_name",
        "country_code",
        "city",
        "cuisines",
        "average_cost_for_two",
        "aggregate_rating",
        "votes",
]

lines = (df1["cuisines"].isin(cuisines)) & (df1["country_code"].isin(countries))

top10 = df1.loc[lines, cols].sort_values(
        ["aggregate_rating", "restaurant_id"], ascending=[False, True]
)


top10 = top10.head(2).reset_index(drop=True)
top10    

,restaurant_id,restaurant_name,country_code,city,cuisines,average_cost_for_two,aggregate_rating,votes
0,7528,Indian Grill Room,India,Gurgaon,North Indian,1800,4.9,3062
1,8913,Pirates of Grill,India,Gurgaon,North Indian,2000,4.9,5760


3 - Top 10 Melhores Tipos de Culinárias

In [133]:
(df1[['cuisines', 'restaurant_id', 'aggregate_rating']]
.groupby(['cuisines'])
.mean()
.sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])
.reset_index()
.head(10)
)

,cuisines,restaurant_id,aggregate_rating
0,Others,16634578.0,4.90
1,Ramen,11447924.0,4.80
2,Ottoman,6003596.0,4.80
3,Egyptian,18741131.5,4.80
4,Sunda,7417452.5,4.75
5,Fresh Fish,11350481.0,4.75
6,Author,6712964.0,4.70
7,Polish,16990065.0,4.70
8,Burmese,17745781.5,4.65
9,Pan Asian,7006871.0,4.60


4 - Top 10 piores tipos de culinária

In [139]:
(df1[['cuisines', 'aggregate_rating']]
.groupby(['cuisines'])
.mean()
.sort_values('aggregate_rating')
.reset_index()
.head(10)
)

,cuisines,aggregate_rating
0,Mineira,0.000000
1,Drinks Only,0.000000
2,nan,1.807692
3,Afghan,2.050000
4,Bengali,2.133333
5,Tea,2.460000
6,Tibetan,2.800000
7,Durban,2.900000
8,Armenian,3.400000
9,Cantonese,3.400000
